In [ ]:
# ============================================
# Phase 2 - Model Comparison Notebook
# YOLOv8n vs RT-DETR-L (Ultralytics)
# ============================================

import os
import time
from tqdm import tqdm
import torch
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# ============================================
# 1️⃣ Environment & Paths
# ============================================
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

BASE_DIR = os.getcwd()
DATASET_DIR = os.path.join(BASE_DIR, "datasets", "caltech", "images", "test")
RESULTS_DIR = os.path.join(BASE_DIR, "reports", "phase2_results")
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"\nDataset: {DATASET_DIR}")
print(f"Results will be saved to: {RESULTS_DIR}")

In [ ]:
# ============================================
# 2️⃣ Load Models
# ============================================
print("\nLoading models...")
yolo_model = YOLO("yolov8n.pt")
rtdetr_model = YOLO("rtdetr-l.pt")
print("✅ Models loaded successfully.")

In [ ]:
# ============================================
# 3️⃣ Define Benchmark Function
# ============================================
def benchmark_dataset(model, name, source_dir, save_dir, max_images=None):
    """Run inference on all images in a folder and return average FPS."""
    image_files = [
        os.path.join(source_dir, f)
        for f in os.listdir(source_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]
    if max_images:
        image_files = image_files[:max_images]

    os.makedirs(save_dir, exist_ok=True)

    total_time = 0
    print(f"\nRunning {name} on {len(image_files)} images...")

    for img_path in tqdm(image_files):
        start = time.time()
        results = model(img_path, verbose=False)
        total_time += time.time() - start

        # Save annotated image
        img_name = os.path.basename(img_path)
        save_path = os.path.join(save_dir, img_name)
        results[0].save(filename=save_path)

    avg_time = total_time / len(image_files)
    fps = 1.0 / avg_time
    print(f"{name} | Avg Time: {avg_time:.3f}s | FPS: {fps:.2f}")
    return avg_time, fps